In [2]:
import pandas as pd
import json
import zipfile
import gzip
import glob
import os

In [7]:
!ls /disk/download/data/ | head

2011-02-12-0.json.gz
2011-02-12-10.json.gz
2011-02-12-11.json.gz
2011-02-12-12.json.gz
2011-02-12-13.json.gz
2011-02-12-14.json.gz
2011-02-12-15.json.gz
2011-02-12-16.json.gz
2011-02-12-17.json.gz
2011-02-12-18.json.gz
ls: write error: Broken pipe


In [17]:
path = '/disk/download/data/2011-02-12-0.json.gz'

with gzip.open(path, 'rb') as f:
    file_content = f.read().decode()
    
    rows = [json.loads(line) for line in file_content.split('\n') if len(line) > 0]

In [19]:
rows[0]

{'repo': {'id': 355634,
  'url': 'https://api.github.dev/repos/projectblacklight/blacklight',
  'name': 'projectblacklight/blacklight'},
 'type': 'PushEvent',
 'org': {'gravatar_id': '6cb76a4a521c36d96a0583e7c45eaf95',
  'id': 120516,
  'url': 'https://api.github.dev/orgs/projectblacklight',
  'avatar_url': 'https://secure.gravatar.com/avatar/6cb76a4a521c36d96a0583e7c45eaf95?d=http://github.dev%2Fimages%2Fgravatars%2Fgravatar-org-420.png',
  'login': 'projectblacklight'},
 'public': True,
 'created_at': '2011-02-12T00:00:00Z',
 'payload': {'shas': [['d3da39ab96a2caecae5d526596a04820c6f848a6',
    'b31a437f57bdf70558bed8ac28790a53e8174b87@stanford.edu',
    "We have to stay with cucumber < 0.10 and rspec < 2, otherwise our tests break. I updated the gem requirements to reflect this, so people won't accidentally upgrade to a gem version that's too new when they run rake gems:install",
    'Bess Sadler'],
   ['898150b7830102cdc171cbd4304b6a783101c3e3',
    'b31a437f57bdf70558bed8ac28790a5

In [20]:
rows[0].keys()

dict_keys(['repo', 'type', 'org', 'public', 'created_at', 'payload', 'actor', 'id'])

In [22]:
!pip3 install treelib

    100% |████████████████████████████████| 829kB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for treelib ... done
  Stored in directory: /home/lspiegel/.cache/pip/wheels/68/1d/92/c50ec52951ccebafb40f3b8f0beb28fbaf745431c14a17c497
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/lspiegel/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built treelib future


In [23]:
import treelib

In [ ]:
# build tree based on keys/attributes


In [26]:
tree = treelib.Tree()

n = tree.create_node("test", 0)

In [28]:
n.identifier

0

In [147]:

def detect_type(value):
    if value is None:
        return 'None'
    elif value == dict:
        return 'dict'
   
    if isinstance(value, dict):
        return 'dict'
    if isinstance(value, list):
        
        if len(value) == 0:
            return 'List[]'
        # check if it's a homogenous list! -> if not type as list[any]
        el_types = {detect_type(el) for el in value}
        if len(el_types) == 1:
            return 'List[{}]'.format(list(el_types)[0])
        
        return 'List[Any]'
    if isinstance(value, str):
        return 'str'
    if isinstance(value, bool):
        return 'bool'
    if isinstance(value, int):
        return 'int'
    if isinstance(value, float):
        return 'float'

    return ''

class TreeData:
    def __init__(self, name, type=None):
        self.name = name
        self.type = type
        self.count = 1
    
    @property
    def desc(self):
        return '{}: {}'.format(self.name, self.type)
    
    def toJSON(self):
        return json.dumps({'type':self.type}, sort_keys=True)
    
def dict_to_tree(d):
    tree = treelib.Tree()
    node = tree.create_node(tag="root", identifier=0, data=TreeData("root", 'dict'))
    
    
    def recursive_helper(tree, parent, d):
        for key, value in d.items():
            node = tree.create_node(key, parent=parent.identifier, data=TreeData(key, detect_type(value)))

            if isinstance(value, dict):
                # recurse!
                recursive_helper(tree, node, value)
            elif isinstance(value, list):
                # recurse!
                # only take first element??
                if len(value) > 0 and isinstance(value[0], dict):
                    recursive_helper(tree, node, value[0])
            else:
                pass
            
    recursive_helper(tree, node, d)
   
    return tree

class ComplexEncoder(json.JSONEncoder):
    def default(self, z):
        if isinstance(z, TreeData):
            return z.toJSON()
        else:
            return super().default(z)

In [149]:
t1 = dict_to_tree(rows[0])
key = json.dumps(t1.to_dict(with_data=True), cls=ComplexEncoder, sort_keys=True)

In [139]:
help(t1.to_json)

Help on method to_json in module treelib.tree:

to_json(with_data=False, sort=True, reverse=False) method of treelib.tree.Tree instance
    To format the tree in JSON format.



In [86]:
dict_to_tree(rows[0]).show(data_property="desc")

root: dict
├── actor: dict
│   ├── avatar_url: str
│   ├── gravatar_id: str
│   ├── id: int
│   ├── login: str
│   └── url: str
├── created_at: str
├── id: str
├── org: dict
│   ├── avatar_url: str
│   ├── gravatar_id: str
│   ├── id: int
│   ├── login: str
│   └── url: str
├── payload: dict
│   ├── actor: str
│   ├── actor_gravatar: str
│   ├── head: str
│   ├── push_id: int
│   ├── ref: str
│   ├── repo: str
│   ├── shas: List[List[str]]
│   └── size: int
├── public: bool
├── repo: dict
│   ├── id: int
│   ├── name: str
│   └── url: str
└── type: str



In [87]:
dict_to_tree(rows[1]).show(data_property="desc")

root: dict
├── actor: dict
│   ├── avatar_url: str
│   ├── gravatar_id: str
│   ├── id: int
│   ├── login: str
│   └── url: str
├── created_at: str
├── id: str
├── payload: dict
│   ├── action: str
│   ├── actor: str
│   ├── actor_gravatar: str
│   └── repo: str
├── public: bool
├── repo: dict
│   ├── id: int
│   ├── name: str
│   └── url: str
└── type: str



In [88]:
t = dict_to_tree(rows[0])

In [89]:
counts = {}

counts[t] = 1

In [91]:
# count different trees and how often they occur!

In [122]:
def compare_trees(t1, t2):
    
    def compare_nodes(tree1, tree2, node1, node2): 
      # reimplement this comparison based on your needs: use data,...
      if(node1.data != node2.data):
        return False
      else:
        # access global trees
        children1 = tree1.children(node1.identifier)
        children2 = tree2.children(node2.identifier)

        if len(children1) != len(children2):
            return False
        elif len(children1) > 0:
            # recusivly compare children
            return all([compare_nodes(tree1, tree2, c1i, c2i) for c1i, c2i in zip(children1, children2)])
        else:
            return True
    
    return compare_nodes(t1, t2, t1.get_node(t1.root), t2.get_node(t2.root))

In [150]:
from tqdm import tqdm

In [155]:
counts = {}

for row in tqdm(rows):
    t = dict_to_tree(row)
    
    key = json.dumps(t.to_dict(with_data=True), cls=ComplexEncoder, sort_keys=True)
    
    if key not in counts.keys():
        counts[key] = (t, 1)
    else:
        t, c = counts[key]
        counts[key] = (t, c + 1)

100%|██████████| 1552/1552 [00:02<00:00, 621.52it/s]


In [165]:
res = sorted(list(counts.items()), key=lambda x: -x[1][1]) 

In [124]:
t1 = dict_to_tree(rows[0])
t2 = dict_to_tree(rows[1])

In [125]:
compare_trees(t1, t2)

False

In [105]:
t1.root

0

In [128]:
t1.to_json(with_data=True)

TypeError: Object of type 'TreeData' is not JSON serializable